In [1]:
from sklearn.datasets import make_blobs
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# 合成データセットの作成
X, y = make_blobs(random_state=0)
# dataトラベルを訓練セットとテストセットに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
# モデルのインスタンスを生成し、訓練データで学習
logreg = LogisticRegression().fit(X_train, y_train)
# テストセットでモデルを学習
print(f"Test set score: {round(logreg.score(X_test, y_test), 3)}")

Test set score: 0.88


In [2]:
# データの分割は、未知のデータに対する汎化性能の検証をしたいから
# より頑健な汎化性能評価である交差検証(cross-validation)を導入する

In [1]:
# 交差検証は汎化性能を評価する統計的手法で、ただ単に訓練セットとテストセットに分割する方法と比べて,
# より安定で徹底した手法である。交差検証ではデータの分割を何度も繰り返して行い、複数のモデルを訓練する。
# k分割交差検証はよく利用される。多くの場合5〜10になる。
# 5分割の場合、まずはデータを5つのおおよそ同じサイズに分割する

In [3]:
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

iris = load_iris()
logreg = LogisticRegression()

scores = cross_val_score(logreg, iris.data, iris.target)
print(f"Cross-Validation score: {scores}")

Cross-Validation score: [0.96078431 0.92156863 0.95833333]


In [4]:
# 分割数を5に変更する
scores = cross_val_score(logreg,  iris.data, iris.target, cv=5)
print(f"Cross-Validation score: {scores}")

Cross-Validation score: [1.         0.96666667 0.93333333 0.9        1.        ]


In [5]:
# 交差検証の精度をまとめるには一般に平均値を求める
print(f"Average cross-validation score: {scores.mean()}")

Average cross-validation score: 0.9600000000000002


In [7]:
# 交差検証の場合、train_test_splitと違って、全てのデータが正確に1度だけテストに用いられる
# モデルはデータセットの全てのサンプルに対して良い汎化性能を示さなければ、交差検証スコアとその平均を高くすることはできない
# 交差検証の最大の問題は計算コストである
# 交差検証の目的は特定のデータセットに対して、どの程度汎化性能ができるかを評価すること

In [8]:
# 層化k分割交差検証と他の戦略

In [11]:
from sklearn.datasets import load_iris

# データが準備されている順番にk分割すると、規則性があって、テストデータとしては好ましい場合がある
iris = load_iris()
print(f"Iris labels: {iris.target}")

Iris labels: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [12]:
# 層化k分割交差検証を用いて、信頼できる汎化性能を推定する

In [13]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5)
# cpパラメータに交差検証分割器を指定することができる
print(f"Cross-validation scores: {cross_val_score(logreg, iris.data, iris.target, cv=kfold)}")

Cross-validation scores: [1.         0.93333333 0.43333333 0.96666667 0.43333333]


In [14]:
kfold = KFold(n_splits=3)
print(f"Cross-validation scores: {cross_val_score(logreg, iris.data, iris.target, cv=kfold)}")

Cross-validation scores: [0. 0. 0.]


In [15]:
# 分割前にデータをシャッフルする
kfold = KFold(n_splits=3, shuffle=True, random_state=0)
print(f"Cross-validation scores: {cross_val_score(logreg, iris.data, iris.target, cv=kfold)}")

Cross-validation scores: [0.9  0.96 0.96]


In [17]:
# 1つ抜き交差検証
from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()
scores = cross_val_score(logreg, iris.data, iris.target, cv=loo)
print(f"Number of cv iterations: {len(scores)}")
print(f"Mean accuracy: {scores.mean()}")

umber of cv iterations: 150
Mean accuracy: 0.9533333333333334


In [18]:
#  シャッフル分割交差検証
# 訓練セットとテストセットのサイズとは独立に繰り返し回数を制御できる
from sklearn.model_selection import ShuffleSplit
shuffle_split = ShuffleSplit(test_size=.5, train_size=.5, n_splits=10)
scores = cross_val_score(logreg, iris.data, iris.target, cv=shuffle_split)
print(f"Cross-validation scores: {scores}")

Cross-validation scores: [0.97333333 0.89333333 0.88       0.96       0.97333333 0.92
 0.96       0.96       0.98666667 0.93333333]


In [22]:
# グループ付き交差検証
from sklearn.model_selection import GroupKFold
from sklearn.datasets import make_blobs
import mglearn

# 合成データセットを作成
X, y = make_blobs(n_samples=12, random_state=0)
# 最初の3サンプルが同じグループに、次の４つが同じグループに、というようにする
groups = [0,0,0,1,1,1,1,2,2,3,3,3]
scores = cross_val_score(logreg, X, y, groups, cv=GroupKFold(n_splits=3))
print(f"Cross-validation scores: {scores}")


Cross-validation scores: [0.75       0.8        0.66666667]
